# Capstone Project - ItemMaster NLP

## 1.  Load Package

In [1]:
from __future__ import division
import nltk
import string
import os
#from nltk.corpus import stopwords
from itertools import combinations
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sys import getsizeof
from autocorrect import spell
import re

## 2. Define Functions

For function find_matching_SFAccountID(row,pdSFAccount_Uniq,thresholdB,thresholdM):

after consulting Item Master's Data Analytics team, we using the following measures to decide whether a match is found:
1. When Brand name in retailer's file finds a larger than 60% (Jaccard similarity > 0.6) match with a Sales Force Account Name, and Sales Force parent account name matches the retailer's manufacture's name in retail file. We say that the match is found! The function call will give a return_values = [AccountName_Brand, Brand_Account _SFAccountID, Brand_Account_SFParentAccountID, brand_similarity].
2. When Brand name in retailer's file finds a larger than 60% (Jaccard similarity > 0.6) match with a Sales Force Account Name, and Sales Force parent account ID is 'nan', We say that the match is found! The function call will give a return_values = [AccountName_Brand, Brand_Account _SFAccountID, Brand_Account_SFParentAccountID, brand_similarity].
3. When Brand name in retailer's file finds a larger than 60% (Jaccard similarity > 0.6) match with a Sales Force Account Name, but Sales Force parent account name doesn't match the retailer's manufacture's name. According to Item Master's analytics team, this should be considered as a match. Additionally a new record created with AccountName_NEW = parent_SFAccountName:retailer_manufacture. The function call will give a return_values = [AccountName_NEW, Brand_Account _SFAccountID, Brand_Account_SFParentAccountID, brand_similarity].
4. 	When Brand name in retailer's file couldn't find a larger than 60% (Jaccard similarity > 0.6) match with a Sales Force Account Name, but the manufacturer name of the product can, We say that the match is found! The function call will give a return_values = [AccountName_Manufacturer, Manufacturer_Account _SFAccountID, Manufacturer_Account_SFParentAccountID, Manufacturer_similarity].



In [19]:
# Check if a token will be null value
def is_empty(word):   
    if word in [",", "inc", ".","company","co","co.","kg","corporation","&","","corp","llc","ltd","plc","kgaa","-","sa",
                "group","/","incorporated","lp","of","partner","the","?","import","'s"]:
        value = 1
    else:
        value = 0                                                               
    return value

def str_After_Colon(str1):
    temp_str1 = str1.rsplit(":")
    if len(temp_str1) == 2:
        str1 = temp_str1[1]
    return str1

# Remove tokens that doesn't add specific meanings to manufcturer or brand
def str_clean(str_tokens):
    str_tokens_c = [word for word in str_tokens if word not in [",", "inc", ".","company","co","co.","kg","corporation","&",
                                                                "corp","llc","ltd","plc","kgaa","-","sa","group","/",
                                                               "incorporated","lp","of","partner","the","?","import","'s"]]                                                            
    return str_tokens_c

# Lemmatize a single tokenized string
def str_tokens_lemmatize(str_tokens):
    wnl = nltk.WordNetLemmatizer()
    str_tokens_l = list(map(lambda x:wnl.lemmatize(x), str_tokens))
    return str_tokens_l

def Spell(x):
    matches_1=re.findall('[0-9]',x)
    matches_2=re.findall('[0-9][0-9]',x)
    if len(matches_1)==0 & len(matches_2)==0:
        x = spell(x)    
    return x

#Auto correct spelling of a single tokenized string
def str_tokens_spelling(str_tokens):
    str_tokens_s = list(map(lambda x:Spell(x), str_tokens))
    return str_tokens_s

def split_numstring(x):
    match_1 = re.match(r"([0-9]+)([a-z]+)", x, re.I)
    match_2 = re.match(r"([a-z]+)([0-9]+)", x, re.I)
    #print (match_1)
    #print(x)
    if match_1:
        items = list(match_1.groups())
    elif match_2:
        items = list(match_2.groups())
    else:
        items = x
    return items

def str_list_group(str_tokens_split):
    str_tokens_new = []
    for item in range(len(str_tokens_split)): 
        if type(str_tokens_split[item]) is str:
            str_tokens_new.append(str_tokens_split[item])
        elif type(str_tokens_split[item]) is list:
            for token in range(len(str_tokens_split[item])):
                str_tokens_new.append(str_tokens_split[item][token])
    return str_tokens_new


#get tokenized, cleaned, lemmatized and spelling correctedstr2. str2 is a list of stings
def get_str2_tokens_cls(str2):
    
    str2_AfterColon = list(map(lambda x:str_After_Colon(x),str2))
    #To lower case str2 and then tokenize it
    str2_lower = list(map(lambda x:x.lower(),str2_AfterColon))
    str2_tokens = list(map(lambda x:nltk.word_tokenize(x), str2_lower))
       
    # Remove tokens that doesn't add specific meanings
    str2_tokens_c = list(map(lambda x:str_clean(x), str2_tokens))
    
    str2_tokens_new = []
    for row_tokens_c in str2_tokens_c:
        row_tokens_split = list(map(lambda x:split_numstring(x), row_tokens_c))
        row_tokens_new = str_list_group(row_tokens_split)
        str2_tokens_new.append(row_tokens_new)
       
    # Lemmatize str2 after they are lower-cased, tokenized, cleaned and numstrings splited     
    str2_tokens_cl = list(map(lambda x:str_tokens_lemmatize(x), str2_tokens_new))
    
    #Auto correct spelling after str2 had been cleaned, tokenized and lemmatized   
    str2_tokens_cls = list(map(lambda x:str_tokens_spelling(x), str2_tokens_cl))
    
    return str2_tokens_cls


###### For function find_matching_SFAccount(row,pdSFAccount_Uniq,str2_tokens_cls,thresholdB,thresholdM):

After consulting Item Master's Data Analytics team, we using the following measures to decide whether a match is found:
1. When Brand name in retailer's file finds a larger than 60% (Jaccard similarity > 0.6) match with a Sales Force Account Name, and Sales Force parent account name matches the retailer's manufacture's name in retail file. We say that the match is found! The function call will give a return_values = [AccountName_Brand, Brand_Account _SFAccountID, Brand_Account_SFParentAccountID, brand_similarity].
2. When Brand name in retailer's file finds a larger than 60% (Jaccard similarity > 0.6) match with a Sales Force Account Name, and Sales Force parent account ID is 'nan', We say that the match is found! The function call will give a return_values = [AccountName_Brand, Brand_Account _SFAccountID, Brand_Account_SFParentAccountID, brand_similarity].
3. When Brand name in retailer's file finds a larger than 60% (Jaccard similarity > 0.6) match with a Sales Force Account Name, but Sales Force parent account name doesn't match the retailer's manufacture's name. According to Item Master's analytics team, this should be considered as a match. Additionally a new record created with AccountName_NEW = parent_SFAccountName:retailer_manufacture. The function call will give a return_values = [AccountName_NEW, Brand_Account _SFAccountID, Brand_Account_SFParentAccountID, brand_similarity].
4. 	When Brand name in retailer's file couldn't find a larger than 60% (Jaccard similarity > 0.6) match with a Sales Force Account Name, but the manufacturer name of the product can, We say that the match is found! The function call will give a return_values = [AccountName_Manufacturer, Manufacturer_Account _SFAccountID, Manufacturer_Account_SFParentAccountID, Manufacturer_similarity].


In [59]:
# Compare two strings using jacard similarity
def jacard_similarity_found(str1,str2_tokens_cls):
    # Takes two strings
    # calculates the intersection and union
    # of the two lists, and returns
    # Jacard similarity value
 
    str1_AfterColon = str_After_Colon(str1)
    str1_tokens = nltk.word_tokenize(str1_AfterColon.lower())
      
    
    # Remove tokens that doesn't add specific meanings to manufcturer or brand   
    str1_tokens_c = str_clean(str1_tokens) 
    
    str1_tokens_split = list(map(lambda x:split_numstring(x), str1_tokens_c))
    str1_tokens_new = str_list_group(str1_tokens_split)
    
    # Lemmatize str1 and str2 after they are lower-cased, tokenized and cleaned   
    str1_tokens_cl = str_tokens_lemmatize(str1_tokens_new)   
    
    #Auto correct spelling after str1 and str2 both been tokenized and lemmatized
    str1_tokens_cls = str_tokens_spelling(str1_tokens_cl)          
       
    # Get the unigram of str1_tokens_s and strs_tokens_s
    str1_unigram = nltk.ngrams(str1_tokens_cls, 1)
    str2_unigram = list(map(lambda x:nltk.ngrams(x,1), str2_tokens_cls))
    
    str1_unigram_set = set(str1_unigram)
    str2_unigram_set = list(map(lambda x:set(x), str2_unigram))
    
    intersection = list(map(lambda x:len(list(str1_unigram_set.intersection(x))), str2_unigram_set))
    
    jacard_similarity_found = []
    for i in range(len(str2_unigram_set)):
        union = len(str1_unigram_set) + len(str2_unigram_set[i]) - intersection[i]
        jacard_similarity = intersection[i] / union
        if (jacard_similarity > 0):
            jacard_similarity_found.append([i,jacard_similarity])  
        i = i+1
    
    if len(jacard_similarity_found) == 0:
        return ['NaN', 0.0]
    else:
        jacard_similarity_found = np.array(jacard_similarity_found)
        ix = np.isin(jacard_similarity_found[:,1], jacard_similarity_found[:,1].max())
        index = int(jacard_similarity_found[ix][0][0])

        return [index, jacard_similarity_found[:,1].max()]
    

def find_matching_SFAccount(row,pdSFAccount_Uniq,str2_tokens_cls,thresholdB,thresholdM): 
    Manufacturer = row[0]
    Brand = row[1]
    Manufacturer = str_After_Colon(Manufacturer)
    Brand = str_After_Colon(Brand)
    if is_empty(Manufacturer.lower()) == 1: 
        return_values = ["No Match","", "", ""]              
        return return_values
    elif is_empty(Brand.lower()) == 1:
        sim_Manufacturer = jacard_similarity_found(Manufacturer, str2_tokens_cls)
        if sim_Manufacturer[1] > thresholdM:
            Manufacturer_Account_Obj = pdSFAccount_Uniq.loc[pdSFAccount_Uniq.index[sim_Manufacturer[0]]]
            SFAccountID = Manufacturer_Account_Obj.SFAccountID
            SFParentAccountID = Manufacturer_Account_Obj.SFParentAccountID
            AccountName = pdSFAccount_Uniq.index[sim_Manufacturer[0]]
            return_values = [AccountName,SFAccountID,SFParentAccountID,sim_Manufacturer[1]]
            return return_values
        else:
            return_values = ["No Match","", "", ""]              
            return return_values        
    else:
        sim_Brand = jacard_similarity_found(Brand, str2_tokens_cls)
        if sim_Brand[1] > thresholdB:
            Brand_Account_Obj = pdSFAccount_Uniq.loc[pdSFAccount_Uniq.index[sim_Brand[0]]]
            Brand_Account_SFParentAccountID = Brand_Account_Obj.SFParentAccountID
            if str(Brand_Account_SFParentAccountID) == 'nan':
                SFAccountID = Brand_Account_Obj.SFAccountID
                AccountName_B = pdSFAccount_Uniq.index[sim_Brand[0]]
                return_values = [AccountName_B, SFAccountID, Brand_Account_SFParentAccountID, sim_Brand[1]]
                return return_values
            else:
                sim_Manufacturer = jacard_similarity_found(Manufacturer, str2_tokens_cls)
                if sim_Manufacturer[1] > thresholdM:
                    Manufacturer_Account_Obj = pdSFAccount_Uniq.loc[pdSFAccount_Uniq.index[sim_Manufacturer[0]]]
                    Manufacturer_Account_SFAccountID = Manufacturer_Account_Obj.SFAccountID
                    if Manufacturer_Account_SFAccountID == Brand_Account_SFParentAccountID:
                        SFAccountID = Brand_Account_Obj.SFAccountID
                        AccountName_B = pdSFAccount_Uniq.index[sim_Brand[0]]
                        return_values = [AccountName_B, SFAccountID, Brand_Account_SFParentAccountID, sim_Brand[1]]
                        return return_values
                    else:
                        for AccountName_M in pdSFAccount_Uniq.index:
                            AccountName_NoColon = str_After_Colon(AccountName_M)
                            Manufacturer_Account_Obj = pdSFAccount_Uniq.loc[AccountName_M]
                            if Manufacturer_Account_Obj.SFAccountID == Brand_Account_SFParentAccountID:
                                AccountName_NEW = AccountName_NoColon + ':' + Manufacturer
                                SFAccountID = Brand_Account_Obj.SFAccountID
                                return_values = [AccountName_NEW, SFAccountID, Brand_Account_SFParentAccountID, sim_Brand[1]]
                                return return_values
                else:
                    for AccountName_M in pdSFAccount_Uniq.index:
                        AccountName_NoColon = str_After_Colon(AccountName_M)
                        Manufacturer_Account_Obj = pdSFAccount_Uniq.loc[AccountName_M]
                        if Manufacturer_Account_Obj.SFAccountID == Brand_Account_SFParentAccountID:
                            AccountName_NEW = AccountName_NoColon + ':' + Manufacturer
                            SFAccountID = Brand_Account_Obj.SFAccountID
                            return_values = [AccountName_NEW, SFAccountID, Brand_Account_SFParentAccountID, sim_Brand[1]]
                            return return_values 
        else:
            sim_Manufacturer = jacard_similarity_found(Manufacturer, str2_tokens_cls)
            if sim_Manufacturer[1] > thresholdM:
                Manufacturer_Account_Obj = pdSFAccount_Uniq.loc[pdSFAccount_Uniq.index[sim_Manufacturer[0]]]
                SFAccountID = Manufacturer_Account_Obj.SFAccountID
                SFParentAccountID = Manufacturer_Account_Obj.SFParentAccountID
                AccountName = pdSFAccount_Uniq.index[sim_Manufacturer[0]]
                return_values = [AccountName,SFAccountID,SFParentAccountID,sim_Manufacturer[1]]
                return return_values
            else:    
                return_values = ["No Match","", "", ""]              
                return return_values
        return_values = ["No Match","", "", ""]              
        return return_values     

In [21]:
PdAccountNames = pd.read_csv('/Users/manqingsun/Desktop/Capstone/Account Names.csv')
PdAccountNames.head(15)

,SFAccountName,SFAccountID,SFParentAccountID
0,ItemMaster,001G000001GoIK2IAN,NaN
1,Pharmavite,001G000001H83JDIAZ,NaN
2,Philips,001G000001H83JEIAZ,NaN
3,"Phillips Foods, Inc.",001G000001H83JFIAZ,NaN
4,Kellogg Company,001G000001H83vHIAR,NaN
5,"Tai Foong USA, Inc.",001G000001H83vIIAR,NaN
6,Zanfel Laboratories Inc,001G000001H83vJIAR,NaN
7,3M,001G000001H83vKIAR,NaN
8,4C Foods Corp.,001G000001H83vLIAR,NaN
9,"AAA Pharmaceutical, Inc.",001G000001H83vMIAR,NaN


##### To avoid job failure, getting the unique accounts from file Account Names.csv. It should be unique.

In [5]:
AccountName_unique = PdAccountNames.SFAccountName.unique()
len(AccountName_unique)

13236

In [6]:
SFAccount_Uniq = []
pdSFAccount_Uniq = pd.DataFrame(SFAccount_Uniq)

In [7]:
pdSFAccount_Uniq['SFAccountName'] = "NA"
pdSFAccount_Uniq['SFAccountID'] = "NA"
pdSFAccount_Uniq['SFParentAccountID'] = "NA"

In [8]:
PdAccountNames.iloc[0]

SFAccountName                ItemMaster
SFAccountID          001G000001GoIK2IAN
SFParentAccountID                   NaN
Name: 0, dtype: object

In [9]:
i = 0
SFAccountName_row = []
DuplicateAcctNames = []
for row in range(len(PdAccountNames)):
    accountname = PdAccountNames.SFAccountName[row]                
    if accountname not in SFAccountName_row:
        SFAccountName_row.append(PdAccountNames.loc[row,"SFAccountName"])
        pdSFAccount_Uniq.loc[i, "SFAccountName"] = PdAccountNames.loc[row,"SFAccountName"]
        pdSFAccount_Uniq.loc[i, "SFAccountID"] = PdAccountNames.loc[row,"SFAccountID"]
        pdSFAccount_Uniq.loc[i, "SFParentAccountID"] = PdAccountNames.loc[row,"SFParentAccountID"]
        i += 1
    else:
        DuplicateAcctNames.append(accountname)
    

In [10]:
len(DuplicateAcctNames)

3

In [11]:
pdDuplicateAcctNames = pd.DataFrame(DuplicateAcctNames)

In [12]:
#write file to csv for future access
pdSFAccount_Uniq.to_csv("/Users/manqingsun/Desktop/Capstone/SFAccount_Uniq.csv")
pdDuplicateAcctNames.to_csv("/Users/manqingsun/Desktop/Capstone/DuplicateAcctNames.csv")

In [13]:
len(pdSFAccount_Uniq)

13236

In [14]:
pdSFAccount_Uniq.head(5)

,SFAccountName,SFAccountID,SFParentAccountID
0,ItemMaster,001G000001GoIK2IAN,NaN
1,Pharmavite,001G000001H83JDIAZ,NaN
2,Philips,001G000001H83JEIAZ,NaN
3,"Phillips Foods, Inc.",001G000001H83JFIAZ,NaN
4,Kellogg Company,001G000001H83vHIAR,NaN


In [15]:
pdSFAccount_Uniq.set_index("SFAccountName", inplace=True)

In [1]:
str2_tokens_cls = get_str2_tokens_cls(pdSFAccount_Uniq.index)

In [2]:
str2_tokens_cls

In [96]:
import csv
rownum = 0
ifile = open('/Users/manqingsun/Desktop/Capstone/Peapod_Retailer_Names.csv', "r") 
reader = csv.reader(ifile)

RetailerAccountNamesArray = ["",""]
RetailerAccountNamesList = []
#rownum = 0


for row in reader:
# Save header row.
    if rownum == 0:
        header = row
    else:
        RetailerAccountNamesArray[0] = row[0]
        RetailerAccountNamesArray[1] = row[1]     
        RetailerAccountNamesList.append(RetailerAccountNamesArray)
        RetailerAccountNamesArray = ["",""]
    rownum += 1
 
ifile.close()

In [97]:
len(RetailerAccountNamesList)

3064

In [98]:
RetailerAccountNamesList[0:15]

[['1 800 FLOWERS.COM INC', 'FANNIE MAY'],
 ['100% NATURAL LLC', 'SANAVI'],
 ['21ST AMENDMENT', '21ST AMENDMENT BREWERY'],
 ['34 DEGREES - FOODS WITH LATITUDE', '34 DEGREES'],
 ['3M COMPANY', '3M'],
 ['3M COMPANY', 'OCELO'],
 ['3M COMPANY', 'POST-IT'],
 ['3M COMPANY', 'SCOTCH'],
 ['4C FOODS CORP', '4C'],
 ['78 RED CO.', '78 RED'],
 ['A FORTUNE FISH GOURMET COMPANY', 'CHEF MARTIN'],
 ['A TO Z WINEWORKS LLC', 'A TO Z'],
 ['A. V. V. WINERY CO', 'TEMPTATION'],
 ['A.M. BRASWELL, JR. FOOD CO., INC.', "BRASWELL'S"],
 ['AAK UK LTD', 'INTERNATIONAL COLLECTION']]

In [99]:
PdRetailerAccountNames = []
PdRetailerAccountNames = pd.DataFrame(data = RetailerAccountNamesList, columns = ["Manufacturer","Brand"] )
PdRetailerAccountNames.head()

,Manufacturer,Brand
0,1 800 FLOWERS.COM INC,FANNIE MAY
1,100% NATURAL LLC,SANAVI
2,21ST AMENDMENT,21ST AMENDMENT BREWERY
3,34 DEGREES - FOODS WITH LATITUDE,34 DEGREES
4,3M COMPANY,3M


In [100]:
# make new column to hold SFAccountName, SFAccountID, SFParentAccountID and AcctName_Similarity
PdRetailerAccountNames['SFAccountName'] = "NA"
PdRetailerAccountNames['SFAccountID'] = "NA"
PdRetailerAccountNames['SFParentAccountID'] = "NA"
PdRetailerAccountNames['AcctName_Similarity'] = "NA"


In [101]:
PdRetailerAccountNames.head()

,Manufacturer,Brand,SFAccountName,SFAccountID,SFParentAccountID,AcctName_Similarity
0,1 800 FLOWERS.COM INC,FANNIE MAY,NA,NA,NA,NA
1,100% NATURAL LLC,SANAVI,NA,NA,NA,NA
2,21ST AMENDMENT,21ST AMENDMENT BREWERY,NA,NA,NA,NA
3,34 DEGREES - FOODS WITH LATITUDE,34 DEGREES,NA,NA,NA,NA
4,3M COMPANY,3M,NA,NA,NA,NA


In [4]:
ReturnValue = []

for i in range(len(RetailerAccountNamesList)):
    row = RetailerAccountNamesList[i]
    ReturnValue = find_matching_SFAccount(row, pdSFAccount_Uniq, str2_tokens_cls, 0.55, 0.3)
    PdRetailerAccountNames.loc[i, 'SFAccountName'] = ReturnValue[0]
    PdRetailerAccountNames.loc[i, 'SFAccountID'] = ReturnValue[1]
    PdRetailerAccountNames.loc[i, 'SFParentAccountID'] = ReturnValue[2]
    PdRetailerAccountNames.loc[i, 'AcctName_Similarity'] = ReturnValue[3]
    print(i)

In [103]:
PdRetailerAccountNames

,Manufacturer,Brand,SFAccountName,SFAccountID,SFParentAccountID,AcctName_Similarity
0,1 800 FLOWERS.COM INC,FANNIE MAY,"Fannie May Confections, Inc",001G000001t1QQHIA2,NaN,0.666667
1,100% NATURAL LLC,SANAVI,Sanavi,001G0000025ZxdpIAC,001G000001qw25YIAQ,1
2,21ST AMENDMENT,21ST AMENDMENT BREWERY,21st Amendment Brewery Company,0014A00002FcfL4QAJ,NaN,1
3,34 DEGREES - FOODS WITH LATITUDE,34 DEGREES,"34 Degrees, LLC.",001G000001S1QS4IAN,NaN,1
4,3M COMPANY,3M,3M,001G000001H83vKIAR,NaN,1
5,3M COMPANY,OCELO,3M,001G000001H83vKIAR,NaN,1
6,3M COMPANY,POST-IT,3M Homecare Division:3M COMPANY,001G0000025ZxY9IAK,001G000001IcYy4IAF,1
7,3M COMPANY,SCOTCH,Scotch Corporation,001G000001akxDuIAI,NaN,1
8,4C FOODS CORP,4C,4C,001G0000025Zw80IAC,001G000001H83vLIAR,1
9,78 RED CO.,78 RED,78 Red Co.,0014A00002GInXAQA1,NaN,1


In [104]:
#write file to csv for future access
PdRetailerAccountNames.to_csv("/Users/manqingsun/Desktop/Capstone/PeapodRetailerAccountNames_w_PredSFAccountInfo041318.csv")